In [28]:
import optuna
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# Specify dtype option on import or set low_memory=False.
pd.options.mode.chained_assignment = None  # default='warn'

import math
import scipy

# Datviz purposes
import matplotlib.pyplot as plt
plt.style.use('bmh')
import seaborn as sns
sns.set_style('darkgrid')
import plotly.express as px
import missingno as msno

import xgboost
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report

from sklearn.model_selection import train_test_split

In [29]:
train_data = pd.read_csv("../4-FeatureEng/train_data.csv", low_memory=False)
test_data = pd.read_csv("../4-FeatureEng/test_data.csv", low_memory=False) # For local
     

In [52]:
# copying the original dataset into X
X=train_data.copy()
# droping building_id and dependent variable damage_grade
# independant variables
X=X.drop(["damage_grade"],axis=1)
# dependent variable
y=train_data["damage_grade"]

In [31]:
from sklearn.preprocessing import MaxAbsScaler
def preprocess_inputs(train_data):
    scale  = MaxAbsScaler()
    train_data = scale.fit_transform(train_data)
    train_data = pd.DataFrame(train_data)
    return train_data

In [32]:
from sklearn.preprocessing import LabelEncoder
def preprocess_targets(train_data):
    le = LabelEncoder()
    train_data = le.fit_transform(train_data)
    return train_data

In [33]:
X = preprocess_inputs(X)
y = preprocess_targets(y)

In [40]:
f = y.copy()

In [34]:

x_train, x_test,y_train, y_test = train_test_split(X,y,test_size = 0.20,random_state = 42 )

In [35]:
def objective(trial):
    param = {
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.05, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0.2, 1.0),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.01, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.1, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 1.0),
        'max_delta_step' : trial.suggest_int('max_delta_step', 1, 3)
        
    }
    model = xgboost.XGBClassifier(**param)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    return f1_score(y_test, y_pred, average='micro')

In [37]:
study = optuna.create_study(direction='maximize', study_name='xgboost')
study.optimize(objective, n_trials=50, show_progress_bar=True)

XgbmPar = study.deepcopy(study.best_params)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2023-04-14 10:12:58,561] A new study created in memory with name: xgboost
d:\anaconda\lib\site-packages\optuna\progress_bar.py:56: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2023-04-14 10:16:24,136] Trial 0 finished with value: 0.4758391462469445 and parameters: {'max_depth': 8, 'learning_rate': 0.5588509450842555, 'n_estimators': 257, 'min_child_weight': 9, 'gamma': 0.4288980097394511, 'subsample': 0.9960085525158514, 'colsample_bytree': 0.6033738024695785, 'reg_alpha': 0.7002909678672343, 'reg_lambda': 0.6513207575491491, 'max_delta_step': 2}. Best is trial 0 with value: 0.4758391462469445.
[I 2023-04-14 10:18:47,861] Trial 1 finished with value: 0.469206462767543 and parameters: {'max_depth': 5, 'learning_rate': 0.06580256912466671, 'n_estimators': 423, 'min_child_weight': 6, 'gamma': 0.26807331151942204, 'subsample': 0.7989971218278891, 'colsample_bytree': 0.27448369485853186, 'reg_alpha': 0.21729820386522186, 'reg_lambda': 0.8345101656138406, 'max_delta_step': 2}. Best is trial 0 with value: 0.4758391462469445.
[I 2023-04-14 10:22:53,254] Trial 2 finished with value: 0.46644905502891554 and parameters: {'max_depth': 9, 'learning_rate': 0.9802952335

AttributeError: 'Study' object has no attribute 'deepcopy'

In [54]:
par = {'max_depth': 9, 'learning_rate': 0.09171368249775058, 'n_estimators': 759, 'min_child_weight': 1, 'gamma': 0.8941301203825416, 'subsample': 0.9235417357436105, 'colsample_bytree': 0.3938564995197662, 'reg_alpha': 0.5575094496034125, 'reg_lambda': 0.21515479375548496, 'max_delta_step': 1}
model = xgboost.XGBClassifier(**par)
model.fit(X, y)

test_data=test_data.drop("id",axis=1)
test_data = preprocess_inputs(test_data)
rf_pred_test_data=model.predict(test_data)

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1 2 3 4], got [1 2 3 4 5]

In [ ]:
submission=pd.read_csv("../datasets/sample_submission.csv")
submission=submission.drop("damage_grade",axis=1)
rf_pred_test_data=pd.DataFrame(rf_pred_test_data)
submission["damage_grade"]=rf_pred_test_data
submission['damage_grade'].unique()
     

array([4, 3, 1, 0, 2], dtype=int64)

In [ ]:
submission.to_csv('submission.csv', index=False)

In [ ]:
test_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77
0,0.222222,0.007007,0.418,0.141414,0.000000,0.0,0.0,0.111111,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0.333333,0.013013,0.396,0.212121,0.000000,0.0,0.0,0.111111,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.333333,0.040040,0.400,0.181818,0.000000,0.0,0.0,0.111111,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.222222,0.025025,0.378,0.202020,0.666667,0.0,0.0,0.111111,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.222222,0.005005,0.375,0.202020,0.333333,0.0,0.0,0.111111,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
f

array([0, 4, 4, ..., 2, 4, 0], dtype=int64)